In [79]:
import warnings
import pandas as pd
import geopandas as gpd
import glob
import numpy as np
# 오류 경고 무시하기
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', None)

In [80]:
import pandas as pd

df = pd.read_csv('./final/melt_1.csv')
df

,자치구,상권_코드,서비스_업종_코드_명,총_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,총_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_상주인구_수,연령대_40_상주인구_수,연령대_50_상주인구_수,연령대_60_이상_상주인구_수,유사_업종_점포_수,총_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수,연령대_50_직장_인구_수,연령대_60_이상_직장_인구_수,집객시설수,교통시설수,당월_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,cluster,geometry,집객력(점수),매출액(점수),경쟁업체(점수),편의성(점수),잠재고객(점수),geometry2
0,관악구,1,PC방,4539089.50,298619.25,1434923.50,955030.25,690792.75,493429.25,666293.25,9588,437,1037,3103,1809,1328,1874,7,1580,1,317,450,449,266,97,0.0,0.0,1.133742e+09,34712409.50,6.263617e+08,3.849777e+08,4.682628e+07,2.976263e+07,11101138.00,0,POLYGON ((126.93440736942736 37.46765303321722...,1,5,5,1,4.333333,LINESTRING (126.93440736942736 37.467653033217...
1,강남구,2,PC방,16273249.00,1207452.75,4389792.25,4411572.75,3071175.25,1708511.75,1484744.50,16458,1503,1557,4477,3368,2134,3419,16,144420,391,31353,46332,37292,21664,7388,658.0,59.0,1.289435e+09,71259296.00,6.372157e+08,3.301747e+08,1.116119e+08,1.020949e+08,27678678.00,0,POLYGON ((127.02248470599385 37.49640036649937...,5,5,5,5,5.000000,LINESTRING (127.02248470599385 37.496400366499...
2,서초구,3,PC방,16273249.00,1207452.75,4389792.25,4411572.75,3071175.25,1708511.75,1484744.50,16458,1503,1557,4477,3368,2134,3419,16,144420,391,31353,46332,37292,21664,7388,658.0,59.0,1.289435e+09,71259296.00,6.372157e+08,3.301747e+08,1.116119e+08,1.020949e+08,27678678.00,0,POLYGON ((127.02248470599385 37.49640036649937...,5,5,5,5,5.000000,LINESTRING (127.02248470599385 37.496400366499...
3,광진구,4,PC방,2924851.00,312811.25,1203307.75,479611.00,325448.00,275166.50,328506.00,4782,209,476,1461,645,500,1491,6,892,5,148,146,178,204,211,83.0,9.0,4.014410e+08,31942312.50,2.537577e+08,6.683056e+07,1.697125e+07,2.961920e+07,2319934.00,0,POLYGON ((127.06601812263857 37.54077443531351...,4,4,4,4,3.333333,LINESTRING (127.06601812263857 37.540774435313...
4,노원구,5,PC방,4589353.75,760577.50,881616.75,663095.00,663288.50,716511.50,904265.25,899,26,101,212,146,163,251,10,1552,9,369,295,343,307,229,190.0,28.0,8.129638e+08,39189698.25,5.329135e+08,1.317742e+08,3.466281e+07,6.180301e+07,12620517.50,0,POLYGON ((127.06000553082049 37.65328168143866...,5,5,5,5,3.000000,LINESTRING (127.06000553082049 37.653281681438...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17738,강남구,229,화초,1146115.50,525059.50,106693.00,86723.75,179014.50,131369.50,117255.00,1319,113,302,145,147,379,233,6,17175,202,3397,3819,3179,4061,2517,0.0,0.0,2.985131e+07,654936.50,3.033390e+06,3.658780e+06,3.949315e+06,5.515253e+06,2378453.00,3,POLYGON ((127.05624428377789 37.49739863928150...,1,2,3,1,3.333333,LINESTRING (127.05624428377789 37.497398639281...
17739,중구,230,화초,856378.50,28532.75,162665.50,241989.00,212396.00,126906.50,83889.00,73,7,3,2,16,7,38,6,30610,14,4275,11589,8473,5661,598,0.0,0.0,2.427971e+08,0.00,6.078499e+06,1.911803e+07,1.702663e+07,5.586503e+06,2266109.25,3,POLYGON ((126.98279757109825 37.56903233158031...,1,5,3,1,2.666667,LINESTRING (126.98279757109825 37.569032331580...
17740,종로구,231,화초,416415.75,28962.75,64605.75,99649.00,99717.25,68454.50,55027.50,279,16,16,45,41,50,111,9,12937,9,1442,4392,4485,2080,529,0.0,0.0,1.581276e+08,0.00,4.344466e+06,2.042287e+07,3.428431e+07,4.708597e+07,15004763.75,3,POLYGON ((126.98050412101318 37.57615189223551...,1,4,4,1,2.333333,LINESTRING (126.98050412101318 37.576151892235...
17741,중구,232,화초,453583.25,33486.25,63577.75,103645.00,101260.50,74842.00,76772.25,1598,123,104,273,301,269,528,9,2190,2,393,736,598,295,166,0.0,0.0,7.796039e+08,0.00,1.109966e+07,1.380959e+07,2.445364e+07,2.932270e+07,46248438.00,3,"POLYGON ((126.9660870334594 37.56157461963889,...",1,5,4,1,2.000000,LINESTRING (126.9660870334594 37.5615746196388...


In [81]:
df_l = df[['자치구', '상권_코드', '서비스_업종_코드_명', '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수', '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수', 'cluster']]

In [82]:
df_r = df[['자치구', '상권_코드', '서비스_업종_코드_명', '연령대_10_상주인구_수', '연령대_20_상주인구_수', '연령대_30_상주인구_수', '연령대_40_상주인구_수', '연령대_50_상주인구_수', '연령대_60_이상_상주인구_수', 'cluster']]

In [83]:
df_w = df[['자치구', '상권_코드', '서비스_업종_코드_명', '연령대_10_직장_인구_수', '연령대_20_직장_인구_수', '연령대_30_직장_인구_수', '연령대_40_직장_인구_수', '연령대_50_직장_인구_수', '연령대_60_이상_직장_인구_수', 'cluster']]

In [84]:
df_f = df[['자치구', '상권_코드', '서비스_업종_코드_명', '집객시설수', '교통시설수', 'cluster']]

In [85]:
df_s = df[['자치구', '상권_코드', '서비스_업종_코드_명', '유사_업종_점포_수', 'cluster']]

In [86]:
df_e = df[['자치구', '상권_코드', '서비스_업종_코드_명', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', 'cluster']]

In [87]:
df_l = df_l.rename(columns = {'연령대_10_유동인구_수' : '10대',
                                             '연령대_20_유동인구_수' : '20대',
                                             '연령대_30_유동인구_수' : '30대',
                                             '연령대_40_유동인구_수' : '40대',
                                             '연령대_50_유동인구_수' : '50대',
                                             '연령대_60_이상_유동인구_수' : '60대 이상'})

In [88]:
df_l = pd.melt(df_l, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '유동인구수')

In [89]:
df_r = df_r.rename(columns = {'연령대_10_상주인구_수' : '10대',
                                             '연령대_20_상주인구_수' : '20대',
                                             '연령대_30_상주인구_수' : '30대',
                                             '연령대_40_상주인구_수' : '40대',
                                             '연령대_50_상주인구_수' : '50대',
                                             '연령대_60_이상_상주인구_수' : '60대 이상'})

In [90]:
df_r = pd.melt(df_r, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '상주인구수')

In [91]:
df_w = df_w.rename(columns = {'연령대_10_직장_인구_수' : '10대',
                                             '연령대_20_직장_인구_수' : '20대',
                                             '연령대_30_직장_인구_수' : '30대',
                                             '연령대_40_직장_인구_수' : '40대',
                                             '연령대_50_직장_인구_수' : '50대',
                                             '연령대_60_이상_직장_인구_수' : '60대 이상'})

In [92]:
df_w = pd.melt(df_w, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '직장인구수')

In [93]:
df_e = df_e.rename(columns = {'연령대_10_매출_금액' : '10대',
                                             '연령대_20_매출_금액' : '20대',
                                             '연령대_30_매출_금액' : '30대',
                                             '연령대_40_매출_금액' : '40대',
                                             '연령대_50_매출_금액' : '50대',
                                             '연령대_60_이상_매출_금액' : '60대 이상'})

In [94]:
df_e = pd.melt(df_e, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '매출금액')

In [95]:
melt_df = pd.merge(df_l, df_r)
melt_df = pd.merge(melt_df, df_w)
melt_df = pd.merge(melt_df, df_f)
melt_df = pd.merge(melt_df, df_s)
melt_df = pd.merge(melt_df, df_e)

In [96]:
melt_df

,자치구,상권_코드,서비스_업종_코드_명,cluster,연령,유동인구수,상주인구수,직장인구수,집객시설수,교통시설수,유사_업종_점포_수,매출금액
0,관악구,1,PC방,0,10대,298619.25,437,1,0.0,0.0,7,3.471241e+07
1,관악구,1,PC방,0,20대,1434923.50,1037,317,0.0,0.0,7,6.263617e+08
2,관악구,1,PC방,0,30대,955030.25,3103,450,0.0,0.0,7,3.849777e+08
3,관악구,1,PC방,0,40대,690792.75,1809,449,0.0,0.0,7,4.682628e+07
4,관악구,1,PC방,0,50대,493429.25,1328,266,0.0,0.0,7,2.976263e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
106453,서초구,233,화초,3,20대,293923.00,537,1553,0.0,0.0,12,6.210168e+05
106454,서초구,233,화초,3,30대,449062.25,1111,3044,0.0,0.0,12,2.797530e+06
106455,서초구,233,화초,3,40대,390172.75,1087,2666,0.0,0.0,12,4.205668e+06
106456,서초구,233,화초,3,50대,314164.75,981,1705,0.0,0.0,12,1.718576e+06


In [97]:
melt_df.to_csv("./final/melt_df_1.csv", index = False)

In [98]:
import pandas as pd

df = pd.read_csv('./final/melt_2.csv')
df

,자치구,상권_코드,서비스_업종_코드_명,총_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,총_상주인구_수,연령대_10_상주인구_수,연령대_20_상주인구_수,연령대_30_상주인구_수,연령대_40_상주인구_수,연령대_50_상주인구_수,연령대_60_이상_상주인구_수,유사_업종_점포_수,총_직장_인구_수,연령대_10_직장_인구_수,연령대_20_직장_인구_수,연령대_30_직장_인구_수,연령대_40_직장_인구_수,연령대_50_직장_인구_수,연령대_60_이상_직장_인구_수,집객시설수,교통시설수,당월_매출_금액,연령대_10_매출_금액,연령대_20_매출_금액,연령대_30_매출_금액,연령대_40_매출_금액,연령대_50_매출_금액,연령대_60_이상_매출_금액,cluster,geometry,집객력(점수),매출액(점수),경쟁업체(점수),편의성(점수),잠재고객(점수),geometry2
0,관악구,1,PC방,4539089.50,298619.25,1434923.50,955030.25,690792.75,493429.25,666293.25,9588,437,1037,3103,1809,1328,1874,7,1580,1,317,450,449,266,97,0.0,0.0,1.133742e+09,34712409.50,6.263617e+08,3.849777e+08,4.682628e+07,2.976263e+07,11101138.00,0,POLYGON ((126.93440736942736 37.46765303321722...,1,5,5,1,4.333333,LINESTRING (126.93440736942736 37.467653033217...
1,강남구,2,PC방,16273249.00,1207452.75,4389792.25,4411572.75,3071175.25,1708511.75,1484744.50,16458,1503,1557,4477,3368,2134,3419,16,144420,391,31353,46332,37292,21664,7388,658.0,59.0,1.289435e+09,71259296.00,6.372157e+08,3.301747e+08,1.116119e+08,1.020949e+08,27678678.00,0,POLYGON ((127.02248470599385 37.49640036649937...,5,5,5,5,5.000000,LINESTRING (127.02248470599385 37.496400366499...
2,서초구,3,PC방,16273249.00,1207452.75,4389792.25,4411572.75,3071175.25,1708511.75,1484744.50,16458,1503,1557,4477,3368,2134,3419,16,144420,391,31353,46332,37292,21664,7388,658.0,59.0,1.289435e+09,71259296.00,6.372157e+08,3.301747e+08,1.116119e+08,1.020949e+08,27678678.00,0,POLYGON ((127.02248470599385 37.49640036649937...,5,5,5,5,5.000000,LINESTRING (127.02248470599385 37.496400366499...
3,광진구,4,PC방,2924851.00,312811.25,1203307.75,479611.00,325448.00,275166.50,328506.00,4782,209,476,1461,645,500,1491,6,892,5,148,146,178,204,211,83.0,9.0,4.014410e+08,31942312.50,2.537577e+08,6.683056e+07,1.697125e+07,2.961920e+07,2319934.00,0,POLYGON ((127.06601812263857 37.54077443531351...,4,4,4,4,3.333333,LINESTRING (127.06601812263857 37.540774435313...
4,노원구,5,PC방,4589353.75,760577.50,881616.75,663095.00,663288.50,716511.50,904265.25,899,26,101,212,146,163,251,10,1552,9,369,295,343,307,229,190.0,28.0,8.129638e+08,39189698.25,5.329135e+08,1.317742e+08,3.466281e+07,6.180301e+07,12620517.50,0,POLYGON ((127.06000553082049 37.65328168143866...,5,5,5,5,3.000000,LINESTRING (127.06000553082049 37.653281681438...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17738,강남구,229,화초,1146115.50,525059.50,106693.00,86723.75,179014.50,131369.50,117255.00,1319,113,302,145,147,379,233,6,17175,202,3397,3819,3179,4061,2517,0.0,0.0,2.985131e+07,654936.50,3.033390e+06,3.658780e+06,3.949315e+06,5.515253e+06,2378453.00,3,POLYGON ((127.05624428377789 37.49739863928150...,1,2,3,1,3.333333,LINESTRING (127.05624428377789 37.497398639281...
17739,중구,230,화초,856378.50,28532.75,162665.50,241989.00,212396.00,126906.50,83889.00,73,7,3,2,16,7,38,6,30610,14,4275,11589,8473,5661,598,0.0,0.0,2.427971e+08,0.00,6.078499e+06,1.911803e+07,1.702663e+07,5.586503e+06,2266109.25,3,POLYGON ((126.98279757109825 37.56903233158031...,1,5,3,1,2.666667,LINESTRING (126.98279757109825 37.569032331580...
17740,종로구,231,화초,416415.75,28962.75,64605.75,99649.00,99717.25,68454.50,55027.50,279,16,16,45,41,50,111,9,12937,9,1442,4392,4485,2080,529,0.0,0.0,1.581276e+08,0.00,4.344466e+06,2.042287e+07,3.428431e+07,4.708597e+07,15004763.75,3,POLYGON ((126.98050412101318 37.57615189223551...,1,4,4,1,2.333333,LINESTRING (126.98050412101318 37.576151892235...
17741,중구,232,화초,453583.25,33486.25,63577.75,103645.00,101260.50,74842.00,76772.25,1598,123,104,273,301,269,528,9,2190,2,393,736,598,295,166,0.0,0.0,7.796039e+08,0.00,1.109966e+07,1.380959e+07,2.445364e+07,2.932270e+07,46248438.00,3,"POLYGON ((126.9660870334594 37.56157461963889,...",1,5,4,1,2.000000,LINESTRING (126.9660870334594 37.5615746196388...


In [99]:
df_l = df[['자치구', '상권_코드', '서비스_업종_코드_명', '연령대_10_유동인구_수', '연령대_20_유동인구_수', '연령대_30_유동인구_수', '연령대_40_유동인구_수', '연령대_50_유동인구_수', '연령대_60_이상_유동인구_수', 'cluster']]

In [100]:
df_r = df[['자치구', '상권_코드', '서비스_업종_코드_명', '연령대_10_상주인구_수', '연령대_20_상주인구_수', '연령대_30_상주인구_수', '연령대_40_상주인구_수', '연령대_50_상주인구_수', '연령대_60_이상_상주인구_수', 'cluster']]

In [101]:
df_w = df[['자치구', '상권_코드', '서비스_업종_코드_명', '연령대_10_직장_인구_수', '연령대_20_직장_인구_수', '연령대_30_직장_인구_수', '연령대_40_직장_인구_수', '연령대_50_직장_인구_수', '연령대_60_이상_직장_인구_수', 'cluster']]

In [102]:
df_f = df[['자치구', '상권_코드', '서비스_업종_코드_명', '집객시설수', '교통시설수', 'cluster']]

In [103]:
df_s = df[['자치구', '상권_코드', '서비스_업종_코드_명', '유사_업종_점포_수', 'cluster']]

In [104]:
df_e = df[['자치구', '상권_코드', '서비스_업종_코드_명', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', 'cluster']]

In [105]:
df_l = df_l.rename(columns = {'연령대_10_유동인구_수' : '10대',
                                             '연령대_20_유동인구_수' : '20대',
                                             '연령대_30_유동인구_수' : '30대',
                                             '연령대_40_유동인구_수' : '40대',
                                             '연령대_50_유동인구_수' : '50대',
                                             '연령대_60_이상_유동인구_수' : '60대 이상'})

In [106]:
df_l = pd.melt(df_l, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '유동인구수')

In [107]:
df_r = df_r.rename(columns = {'연령대_10_상주인구_수' : '10대',
                                             '연령대_20_상주인구_수' : '20대',
                                             '연령대_30_상주인구_수' : '30대',
                                             '연령대_40_상주인구_수' : '40대',
                                             '연령대_50_상주인구_수' : '50대',
                                             '연령대_60_이상_상주인구_수' : '60대 이상'})

In [108]:
df_r = pd.melt(df_r, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '상주인구수')

In [109]:
df_w = df_w.rename(columns = {'연령대_10_직장_인구_수' : '10대',
                                             '연령대_20_직장_인구_수' : '20대',
                                             '연령대_30_직장_인구_수' : '30대',
                                             '연령대_40_직장_인구_수' : '40대',
                                             '연령대_50_직장_인구_수' : '50대',
                                             '연령대_60_이상_직장_인구_수' : '60대 이상'})

In [110]:
df_w = pd.melt(df_w, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '직장인구수')

In [111]:
df_e = df_e.rename(columns = {'연령대_10_매출_금액' : '10대',
                                             '연령대_20_매출_금액' : '20대',
                                             '연령대_30_매출_금액' : '30대',
                                             '연령대_40_매출_금액' : '40대',
                                             '연령대_50_매출_금액' : '50대',
                                             '연령대_60_이상_매출_금액' : '60대 이상'})

In [112]:
df_e = pd.melt(df_e, id_vars = ['자치구', '상권_코드', '서비스_업종_코드_명', 'cluster'],
                      var_name = '연령', value_name = '매출금액')

In [113]:
melt_df = pd.merge(df_l, df_r)
melt_df = pd.merge(melt_df, df_w)
melt_df = pd.merge(melt_df, df_f)
melt_df = pd.merge(melt_df, df_s)
melt_df = pd.merge(melt_df, df_e)

In [114]:
melt_df

,자치구,상권_코드,서비스_업종_코드_명,cluster,연령,유동인구수,상주인구수,직장인구수,집객시설수,교통시설수,유사_업종_점포_수,매출금액
0,관악구,1,PC방,0,10대,298619.25,437,1,0.0,0.0,7,3.471241e+07
1,관악구,1,PC방,0,20대,1434923.50,1037,317,0.0,0.0,7,6.263617e+08
2,관악구,1,PC방,0,30대,955030.25,3103,450,0.0,0.0,7,3.849777e+08
3,관악구,1,PC방,0,40대,690792.75,1809,449,0.0,0.0,7,4.682628e+07
4,관악구,1,PC방,0,50대,493429.25,1328,266,0.0,0.0,7,2.976263e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
106453,서초구,233,화초,3,20대,293923.00,537,1553,0.0,0.0,12,6.210168e+05
106454,서초구,233,화초,3,30대,449062.25,1111,3044,0.0,0.0,12,2.797530e+06
106455,서초구,233,화초,3,40대,390172.75,1087,2666,0.0,0.0,12,4.205668e+06
106456,서초구,233,화초,3,50대,314164.75,981,1705,0.0,0.0,12,1.718576e+06


In [115]:
melt_df.to_csv("./final/melt_df_2.csv", index = False)